# Clasificación binaria: 1 neurona con 1 sola columna en X

In [1]:
import numpy as np

# 1. Datos de ejemplo (ficticios)
# Podrían ser pesos en kg, y = 0 o 1 según alguna condición
X = np.array([30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95], dtype=float)
y = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=float)

## 1. Definir una neurona

**Función de activación**: sigmoide

La neurona hace la siguiente operación:

$$
z^{(i)} = w \cdot x^{(i)} + b
$$
$$
\hat{y}^{(i)} = \sigma(z^{(i)}) = \frac{1}{1 + e^{-z^{(i)}}}
$$

donde $\sigma$ es la **función sigmoide**.

**Forward pass**

Vamos a definir una función `forward` que recibe:

- `X`: vector (N,) de ejemplos, cada valor es un “peso” (o característica)  
- `w`: un escalar (el peso de la neurona)  
- `b`: un escalar (el sesgo)

Devuelve el valor de la **neurona** $\hat{y}$ para cada ejemplo, aplicando sigmoide:

In [79]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def forward(X, w, b):
    """
    Aplica la neurona con sigmoide a todos los datos X.
    X: shape (N,)
    w, b: escalares
    Return: y_pred (N,) con valores entre 0 y 1
    """
    z = w * X + b            # z^{(i)} = w * x^{(i)} + b
    y_pred = sigmoid(z)      # Aplicamos la sigmoide
    return y_pred

# 3. Función de costo: Entropía Cruzada Binaria (BCE)

Para clasificación binaria, se suele usar la **binary cross entropy** (logistic loss).

La Binary Cross Entropy (BCE), también conocida como Log Loss, mide la diferencia entre las etiquetas reales y las probabilidades predichas por el modelo.

$$
\text{BCE} = -\frac{1}{N} \sum_{i=1}^N \Bigl[y^{(i)}\log\bigl(\hat{y}^{(i)}\bigr) + \bigl(1 - y^{(i)}\bigr)\log\bigl(1 - \hat{y}^{(i)}\bigr)\Bigr].
$$


> En muchos casos, se aplica `np.clip` para evitar un log(0), lo que produce `-inf`.

También se llama Log Loss porque utiliza el logaritmo de las probabilidades y penaliza mucho más las predicciones incorrectas cuando el modelo tiene alta confianza en una clase equivocada.

¿Por qué usar BCE y no accuracy o f1_score? Las métricas como accuracy o F1-score se usan para evaluar el rendimiento del modelo, pero no para optimizarlo. La razón es que estas métricas no tienen propiedades matemáticas que permitan calcular gradientes de manera efectiva para la optimización, es decir, no son diferenciables.

In [80]:
def binary_cross_entropy(y_pred, y_true):
    """
    y_pred: (N,) - Valores en [0, 1] predichos por la sigmoide
    y_true: (N,) - 0 o 1 (etiquetas reales)
    """
    # Para evitar problemas de log(0), se acostumbra a añadir un pequeño epsilon
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

## 4. Gradientes para w y b

### 4.1 Derivada parcial de la BCE respecto a w

Recordando que:
$$
\hat{y}^{(i)} = \sigma(z^{(i)}), \quad z^{(i)} = w x^{(i)} + b,
$$
$$
\frac{\partial}{\partial z} \text{BCE} = \hat{y}^{(i)} - y^{(i)},
$$
(cuando se trata de la entropía cruzada con sigmoide)

entonces:

$$
\frac{\partial}{\partial w} \text{BCE} 
= \frac{1}{N} \sum_{i=1}^N (\hat{y}^{(i)} - y^{(i)}) \cdot \frac{\partial z^{(i)}}{\partial w}
= \frac{1}{N} \sum_{i=1}^N (\hat{y}^{(i)} - y^{(i)}) \cdot x^{(i)}.
$$

### 4.2 Derivada parcial de la BCE respecto a b

$$
\frac{\partial}{\partial b} \text{BCE}
= \frac{1}{N} \sum_{i=1}^N (\hat{y}^{(i)} - y^{(i)}) \cdot \frac{\partial z^{(i)}}{\partial b}
= \frac{1}{N} \sum_{i=1}^N (\hat{y}^{(i)} - y^{(i)}) \cdot 1,
$$
porque $\frac{\partial}{\partial b}(b)=1$.

In [81]:
def grad_w(X, y, w, b):
    """
    Calcula la derivada de la BCE wrt w (escalares).
    """
    y_pred = forward(X, w, b)            # (N,)
    # (y_pred - y): (N,)
    # Multiplicamos elemento a elemento por X -> (N,)
    # y hacemos la media
    return np.mean((y_pred - y) * X)

def grad_b(X, y, w, b):
    y_pred = forward(X, w, b)
    # Solo (y_pred - y), y su media
    return np.mean(y_pred - y)

## 5. Entrenamiento (Descenso por gradiente)

Ahora unimos todo en un **bucle de entrenamiento**.

- Tras el entrenamiento, la neurona va a **aproximarse** a valores de $ w $ y $ b $ que **maximicen** la verosimilitud de la clasificación (equivalentemente, **minimicen** la entropía cruzada).
- `y_pred_final` nos dará probabilidades cercanas a 0 o 1.  
  - Para tomar una **decisión binaria** final, podríamos hacer `pred_class = (y_pred_final >= 0.5).astype(int)`.

In [ ]:
np.random.seed(42)
w = np.random.randn() * 0.02  # un valor aleatorio pequeño
b = np.random.randn() * 0.02

learning_rate = 0.06
num_epochs = 7000

for epoch in range(num_epochs):
    
    y_pred = forward(X, w, b)
    loss_value = binary_cross_entropy(y_pred, y)
    
    dw = grad_w(X, y, w, b)
    db = grad_b(X, y, w, b)
    
    w -= learning_rate * dw
    b -= learning_rate * db
    
    # (Opcional) imprimir cada cierto número de épocas
    if (epoch+1) % 400 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss_value:.6f}, w: {w:.4f}, b: {b:.4f}")

# 6. Revisar resultados finales
print("\nEntrenamiento terminado.")
print(f"Parámetros finales: w = {w:.4f}, b = {b:.4f}")

# 7. Predicción final
y_pred_final = forward(X, w, b)
print("Predicciones finales (probabilidades):", y_pred_final)
y_pred_classes = (y_pred_final >= 0.5).astype(int)
print("Predicciones (clases):", y_pred_classes)
print("Valores reales (clase):               ", y)

Epoch 400/5000, Loss: 0.628088, w: 0.1355, b: -0.5517
Epoch 800/5000, Loss: 5.873374, w: 0.1659, b: -1.0899
Epoch 1200/5000, Loss: 3.089710, w: 0.0257, b: -1.6168
Epoch 1600/5000, Loss: 6.039287, w: 0.1994, b: -2.1302
Epoch 2000/5000, Loss: 6.116875, w: 0.2153, b: -2.6213
Epoch 2400/5000, Loss: 0.361202, w: 0.1126, b: -3.1035
Epoch 2800/5000, Loss: 1.343982, w: -0.0137, b: -3.5810
Epoch 3200/5000, Loss: 0.803060, w: -0.0135, b: -4.0401
Epoch 3600/5000, Loss: 0.856839, w: -0.0023, b: -4.4902
Epoch 4000/5000, Loss: 0.531622, w: 0.0138, b: -4.9205
Epoch 4400/5000, Loss: 0.526385, w: 0.0233, b: -5.3326
Epoch 4800/5000, Loss: 0.565527, w: 0.0312, b: -5.7401

Entrenamiento terminado.
Parámetros finales: w = 0.0352, b = -5.9428
Predicciones finales (probabilidades): [0.00747979 0.00890461 0.01059794 0.01260918 0.01499633 0.01782725
 0.02118108 0.02514971 0.02983926 0.03537152 0.0418852  0.04953676
 0.05850077 0.06896917]
Predicciones (clases): [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Valores reales (cla

In [83]:
def accuracy_score(y_true, y_pred):
    # Convertir predicciones a clases binarias (0 o 1) si son probabilidades
    y_pred_classes = (y_pred >= 0.5).astype(int)
    
    # Calcular el número de aciertos
    correct_predictions = np.sum(y_true == y_pred_classes)
    
    # Calcular accuracy
    accuracy = correct_predictions / len(y_true)
    return accuracy

In [84]:
accuracy_value = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy_value:.2%}")

Accuracy: 71.43%
